In [1]:
using Distributed
# add processors if we don't have enough
if length(workers()) < 6
	addprocs(6)
end

@everywhere begin
	using Distributions
	using Plots
	using Turing
end


# Eight school problem

In [2]:
@everywhere begin
	n_schools = 8
	y = [28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0] # estimated treatment effects
	σ = [15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0] # standard error of the effect estimate
end

In [3]:
@everywhere begin
	
	@model function schools(y, σ)
		μ ~ Normal(0, 5)
		τ ~ truncated(Cauchy(0, 5), lower = 0)
		θ ~ filldist(Normal(μ, τ), n_schools)
		for i in 1:n_schools
			y[i] ~ Normal(θ[i], σ[i])
		end
	end

	@model function school_reparam(y, σ)
		μ ~ Normal(0, 5)
		τ ~ Cauchy(0, 5)
		θ ~ filldist(Normal(0, 1), n_schools)
		# y .~ ForEach(μ .+ τ .* θ, σ)
		for i in 1:n_schools
			y[i] ~ Normal(μ + τ * θ[i], σ[i])
		end
	end
end


## NUTS 

In [4]:
c1 = sample(schools(y, σ), NUTS(0.8),MCMCDistributed(), 10_000,6)

Sampling (6 processes)   0%|                            |  ETA: N/A


      From worker 6:	┌ Info: Found initial step size
      From worker 6:	└   ϵ = 0.4
      From worker 2:	┌ Info: Found initial step size
      From worker 2:	└   ϵ = 0.8
      From worker 3:	┌ Info: Found initial step size
      From worker 3:	└   ϵ = 0.8015625000000001
      From worker 4:	┌ Info: Found initial step size
      From worker 4:	└   ϵ = 0.20625000000000002
      From worker 5:	┌ Info: Found initial step size
      From worker 5:	└   ϵ = 0.4
      From worker 7:	┌ Info: Found initial step size
      From worker 7:	└   ϵ = 0.8


Sampling (6 processes)  17%|████▋                       |  ETA: 0:02:58
Sampling (6 processes)  33%|█████████▍                  |  ETA: 0:01:11
Sampling (6 processes)  50%|██████████████              |  ETA: 0:00:36
Sampling (6 processes)  67%|██████████████████▋         |  ETA: 0:00:18
Sampling (6 processes)  83%|███████████████████████▍    |  ETA: 0:00:07
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:43
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:44


Chains MCMC chain (10000×22×6 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 6
Samples per chain = 10000
Wall duration     = 31.0 seconds
Compute duration  = 136.6 seconds
parameters        = μ, τ, θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse    ess_bulk     ess_tail      rhat  ⋯
      Symbol   Float64   Float64   Float64     Float64      Float64   Float64  ⋯

           μ    4.4073    3.3571    0.0672   2674.1849    1646.2361    1.0022  ⋯
           τ    3.9481    3.1164    0.0580   1313.5621     596.3142    1.0086  ⋯
        θ[1]    6.3553    5.7360    0.0756   4701.8715    1689.6166    1.0024  ⋯
        θ[2]    5.0179    4.8360    0.0707   4572.4926    1753.8657    1.0018  ⋯
        θ[3]    3

In [5]:
c2 = sample(school_reparam(y, σ), NUTS(0.65),MCMCDistributed(), 10_000,6)

Sampling (6 processes)   0%|                            |  ETA: N/A


      From worker 2:	┌ Info: Found initial step size
      From worker 2:	└   ϵ = 1.6
      From worker 5:	┌ Info: Found initial step size
      From worker 5:	└   ϵ = 1.6
      From worker 7:	┌ Info: Found initial step size
      From worker 7:	└   ϵ = 1.6
      From worker 6:	┌ Info: Found initial step size
      From worker 6:	└   ϵ = 1.6
      From worker 4:	┌ Info: Found initial step size
      From worker 4:	└   ϵ = 3.2
      From worker 3:	┌ Info: Found initial step size
      From worker 3:	└   ϵ = 1.6


Sampling (6 processes)  17%|████▋                       |  ETA: 0:01:10
Sampling (6 processes)  33%|█████████▍                  |  ETA: 0:00:28
Sampling (6 processes)  50%|██████████████              |  ETA: 0:00:14
Sampling (6 processes)  67%|██████████████████▋         |  ETA: 0:00:07
Sampling (6 processes)  83%|███████████████████████▍    |  ETA: 0:00:03
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:14
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:14


Chains MCMC chain (10000×22×6 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 6
Samples per chain = 10000
Wall duration     = 12.0 seconds
Compute duration  = 68.34 seconds
parameters        = μ, τ, θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse      ess_bulk     ess_tail      rha ⋯
      Symbol   Float64   Float64   Float64       Float64      Float64   Float6 ⋯

           μ    4.4191    3.2928    0.0115    82821.4608   44540.5366    1.000 ⋯
           τ   -0.0520    4.7770    0.0240    43571.8120   36370.9743    1.000 ⋯
        θ[1]   -0.0036    1.0354    0.0037    78749.9468   49653.6080    1.000 ⋯
        θ[2]    0.0003    0.9408    0.0031    92449.2172   44614.2953    1.000 ⋯
        θ[3]   -0

## Using MicroCanonical HMC

In [6]:
@everywhere begin
	using MicroCanonicalHMC
	n_adapts = 1_000 # adaptation steps
	tev = 0.01 # target energy variance
	mchmc = MCHMC(n_adapts, tev; adaptive = true)
end

In [7]:
c3 = sample(schools(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

Sampling (6 processes)   0%|                            |  ETA: N/A


      From worker 6:	[ Info: Tuning eps ⏳
      From worker 6:	[ Info: Tuning L ⏳
      From worker 6:	[ Info: Tuning sigma ⏳
      From worker 5:	[ Info: Tuning eps ⏳
      From worker 5:	[ Info: Tuning L ⏳
      From worker 5:	[ Info: Tuning sigma ⏳
      From worker 4:	[ Info: Tuning eps ⏳
      From worker 4:	[ Info: Tuning L ⏳
      From worker 4:	[ Info: Tuning sigma ⏳
      From worker 3:	[ Info: Tuning eps ⏳
      From worker 3:	[ Info: Tuning L ⏳
      From worker 3:	[ Info: Tuning sigma ⏳
      From worker 7:	[ Info: Tuning eps ⏳
      From worker 7:	[ Info: Tuning L ⏳
      From worker 7:	[ Info: Tuning sigma ⏳
      From worker 2:	[ Info: Tuning eps ⏳
      From worker 2:	[ Info: Tuning L ⏳
      From worker 2:	[ Info: Tuning sigma ⏳
Tuning:   0%|▏                                          |  ETA: 0:11:48
      From worker 6:	  ϵ:     0.9504123238342497
      From worker 6:	  L:     3.1622776601683795
Tuning:   0%|▏                                          |  ETA: 0:11:44
  

Sampling (6 processes)  17%|████▋                       |  ETA: 0:01:22
Sampling (6 processes)  33%|█████████▍                  |  ETA: 0:00:33
Sampling (6 processes)  50%|██████████████              |  ETA: 0:00:17
Sampling (6 processes)  67%|██████████████████▋         |  ETA: 0:00:08
Sampling (6 processes)  83%|███████████████████████▍    |  ETA: 0:00:03
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:16
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:16


Chains MCMC chain (10000×11×6 Array{Float64, 3}):

Iterations        = 1:1:10000
Number of chains  = 6
Samples per chain = 10000
Wall duration     = 12.81 seconds
Compute duration  = 71.97 seconds
parameters        = μ, τ, θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8]
internals         = lp

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

           μ    4.4216    3.2904    0.0744   1957.1529   3025.7585    1.0064   ⋯
           τ    3.6927    3.1475    0.0997    580.7647    524.4950    1.0358   ⋯
        θ[1]    6.1505    5.1471    0.1181   2089.2165   1986.0303    1.0048   ⋯
        θ[2]    4.9676    4.1778    0.0835   2491.0533   3740.3898    1.0062   ⋯
        θ[3]    3.9689    4.7155    0.1086   2002.9439   1857.0600    1.0052   ⋯
        θ[4]    4.7539    4.2977    0.0892   2316.5325   3343.4999    1.0089   ⋯
        θ[5]    3.6085    4.2307    0.

In [8]:
c4 = sample(school_reparam(y, σ), externalsampler(mchmc),MCMCDistributed(), 10_000,6)

Sampling (6 processes)   0%|                            |  ETA: N/A


      From worker 7:	[ Info: Tuning eps ⏳
      From worker 7:	[ Info: Tuning L ⏳
      From worker 7:	[ Info: Tuning sigma ⏳
      From worker 3:	[ Info: Tuning eps ⏳
      From worker 3:	[ Info: Tuning L ⏳
      From worker 3:	[ Info: Tuning sigma ⏳
      From worker 4:	[ Info: Tuning eps ⏳
      From worker 4:	[ Info: Tuning L ⏳
      From worker 4:	[ Info: Tuning sigma ⏳
      From worker 5:	[ Info: Tuning eps ⏳
      From worker 5:	[ Info: Tuning L ⏳
      From worker 5:	[ Info: Tuning sigma ⏳
      From worker 2:	[ Info: Tuning eps ⏳
      From worker 2:	[ Info: Tuning L ⏳
      From worker 2:	[ Info: Tuning sigma ⏳
      From worker 6:	[ Info: Tuning eps ⏳
      From worker 6:	[ Info: Tuning L ⏳
      From worker 6:	[ Info: Tuning sigma ⏳
Tuning:  30%|████████████▋                              |  ETA: 0:00:00
      From worker 6:	
      From worker 6:	
Tuning:  58%|█████████████████████████▏                 |  ETA: 0:00:00
      From worker 6:	
      From worker 6:	
Tuning:  76%

Sampling (6 processes)  17%|████▋                       |  ETA: 0:00:26
Sampling (6 processes)  33%|█████████▍                  |  ETA: 0:00:10
Sampling (6 processes)  50%|██████████████              |  ETA: 0:00:05
Sampling (6 processes)  67%|██████████████████▋         |  ETA: 0:00:03
Sampling (6 processes)  83%|███████████████████████▍    |  ETA: 0:00:01
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:05
Sampling (6 processes) 100%|████████████████████████████| Time: 0:00:05


Chains MCMC chain (10000×11×6 Array{Float64, 3}):

Iterations        = 1:1:10000
Number of chains  = 6
Samples per chain = 10000
Wall duration     = 3.52 seconds
Compute duration  = 20.5 seconds
parameters        = μ, τ, θ[1], θ[2], θ[3], θ[4], θ[5], θ[6], θ[7], θ[8]
internals         = lp

Summary Statistics
  parameters      mean       std      mcse     ess_bulk     ess_tail      rhat ⋯
      Symbol   Float64   Float64   Float64      Float64      Float64   Float64 ⋯

           μ    4.4106    3.3670    0.0234   20798.3977   26653.5281    1.0005 ⋯
           τ   -0.0534    5.0446    0.0396   17438.1629   18384.3757    1.0011 ⋯
        θ[1]   -0.0048    1.0557    0.0070   22920.1205   29995.9002    1.0009 ⋯
        θ[2]   -0.0029    0.9270    0.0063   21646.7270   27388.0114    1.0043 ⋯
        θ[3]    0.0001    1.0121    0.0070   21184.3020   29160.8326    1.0081 ⋯
        θ[4]   -0.0002    0.9290    0.0067   19082.7026   25180.4886    1.0023 ⋯
        θ[5]   -0.0006    0.9406    0.00

In [9]:
using StatsPlots

In [14]:
# plot(c1)

In [15]:
# plot(c2)

In [16]:
# plot(c3)

In [17]:
# plot(c4)

In [ ]:
@everywhere begin
	using AdvancedMH
	rwmh = AdvancedMH.RWMH(10)
end

In [ ]:
school

In [ ]:
c5 = sample(schools(y, σ), externalsampler(rwmh), MCMCDistributed(), 150_000, 6)

In [ ]:
plot(c5)

In [ ]:
c6 = sample(school_reparam(y, σ), externalsampler(rwmh), MCMCDistributed(), 100_000, 6)

In [ ]:
plot(c6)    